In [2]:
import numpy as np
import tensorflow as tf
import autograd
from autograd import numpy as anp

In [3]:
from matplotlib import pyplot as plt
%matplotlib inline

In [10]:
dim = 3
foo = tf.constant(np.random.random((dim, dim)))
tf.reduce_sum(foo)

<tf.Tensor: shape=(), dtype=float64, numpy=3.7743662423905597>

In [70]:
def normal_lp_suff(mu, mu2, tau, log_tau, xsum, x2sum, num_obs):
    lp = \
        -0.5 * tau * ( \
            tf.reduce_sum(x2sum) - \
            2.0 * tf.tensordot(xsum, mu_true, 1) + \
            tf.reduce_sum(mu2)) + \
        0.5 * num_obs * log_tau
    return lp

In [79]:
dim = 3
num_obs = 10000

mu_true = np.arange(dim, dtype=np.float64)
sd_true = 0.5
tau_true = 1 / sd_true ** 2
print(tau_true)

x = np.random.normal(loc=mu_true, scale=sd_true, size=(num_obs, dim))
xsum = np.sum(x, axis=0)
x2sum = x.T @ x

muhat = xsum / num_obs
covhat = x2sum / num_obs - np.outer(muhat, muhat) 
tauhat = 1 / np.mean(np.diag(covhat))
print(tauhat)

4.0
3.9954520978456123


In [89]:
normal_lp_suff(
    mu=mu_true,
    mu2=np.outer(mu_true, mu_true),
    tau=tau_true,
    log_tau=np.log(tau_true),
    xsum=xsum,
    x2sum=x2sum,
    num_obs=num_obs)

<tf.Tensor: shape=(), dtype=float64, numpy=11152.047848957089>

In [94]:
def normal_lp(par, data):
    mu = tf.convert_to_tensor(par['mu'], dtype=tf.float64)
    tau = tf.convert_to_tensor(par['tau'], dtype=tf.float64)
    return normal_lp_suff(
        mu=mu,
        mu2=tf.tensordot(mu, mu, 0),
        tau=tau,
        log_tau=tf.math.log(tau),
        **data)

data = { 'xsum': xsum, 'x2sum': x2sum, 'num_obs': num_obs}
par = { 'mu': mu_true, 'tau': tau_true }
normal_lp(par, data)

<tf.Tensor: shape=(), dtype=float64, numpy=11152.047848957089>

In [121]:
def flatten_par(par):
    tau = tf.convert_to_tensor(par['tau'], dtype=tf.float64)
    tau = tf.reshape(tau, (1, ))
    return tf.concat([par['mu'], tf.math.log(tau) ], axis=0)

def fold_par(par):
    mu_dim = len(par) - 1
    return { 'mu': par[0:mu_dim], 'tau': tf.math.exp(par[mu_dim]) } 

print(par['tau'])
par_flat = flatten_par(par)
par_fold = fold_par(par_flat)
normal_lp(par_fold, data)

4.0


<tf.Tensor: shape=(), dtype=float64, numpy=11152.047848957089>